In [1]:
import requests 
from tqdm.notebook import tqdm

import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def user_df(api_key, access, match_type):
    headers = {'accept': 'application/json',
               'x-nxopen-api-key': api_key}
    # 매치기록 조회를 위한 파라미터
    match_params = {'matchtype' : match_type, 'offset' : 0, 'limit' : 100}
    match_record_url = requests.get(f'https://open.api.nexon.com/fconline/v1/user/match?ouid={access}', params=match_params, headers = headers)

    match_record_parsed_data = match_record_url.json()
    match_user = pd.DataFrame()
    
    for match_id in tqdm(match_record_parsed_data):
        # 매치 상세정보 조회를 위한 코드
        match_detail_url = requests.get(f'https://open.api.nexon.com/fconline/v1/match-detail?matchid={match_id}', headers = headers)
        match_detail_parsed_data = match_detail_url.json()
        temp = pd.DataFrame(match_detail_parsed_data)
        match_user = pd.concat([match_user, temp], axis = 0)
 
    # DTO 정보 리스트 형태로 만들기
    ouid_list = [i.get('ouid') for i in match_user['matchInfo']]
    nickname_list = [i.get('nickname') for i in match_user['matchInfo']]
    matchDetail_list = [i.get('matchDetail') for i in match_user['matchInfo']]
    shoot_list = [i.get('shoot') for i in match_user['matchInfo']]
    pass_list = [i.get('pass') for i in match_user['matchInfo']]
    defence_list = [i.get('defence') for i in match_user['matchInfo']]

    # 매치 정보, 상세정보, 슛, 패스, 수비 등의 정보를 위한 데이터프레임 생성
    matchInfo_data = match_user.iloc[:, :3].reset_index(drop=True)
    ouid_data = pd.DataFrame(ouid_list)
    nickname_data = pd.DataFrame(nickname_list)
    matchDetail_data = pd.DataFrame(matchDetail_list)
    shoot_data = pd.DataFrame(shoot_list)
    pass_data = pd.DataFrame(pass_list)
    defence_data = pd.DataFrame(defence_list)

    ouid_data.rename(columns={0:'ouid'}, inplace=True)
    nickname_data.rename(columns={0:'nickname'}, inplace=True)

    user = pd.concat([matchInfo_data, ouid_data, nickname_data, matchDetail_data, shoot_data, pass_data, defence_data], axis = 1)
    return user

In [ ]:
api_key = "api_key"
access = "access"
match_type = 52

match = user_df(api_key, access, match_type)
match.to_csv('C:\\Users\\이진호\\Desktop\\Programming\\프로젝트\\MyProject2\\match.csv', index=False, encoding='euc-kr')

  0%|          | 0/100 [00:00<?, ?it/s]